NOTE: To work with any of our notebooks on Deepnote, you must first
- "Duplicate" the project into your own Deepnote account (see the button in the top right of the GUI), then 
- click "Move this file to notebooks".  

Then it should run!

# Writing (simple) systems in Drake

Drake is a powerful modeling language for authoring models of dynamical systems.  The goal of this exercise is to learn how to author a very simple dynamical systems.  To start, take a few minutes to work through [this Drake tutorial](https://deepnote.com/workspace/Drake-0b3b2c53-a7ad-441b-80f8-bf8350752305/project/Tutorials-2b4fc509-aef2-417d-a40d-6071dfed9199/%2Fdynamical_systems.ipynb).

Your first task is to write a simple *continuous-time* system that implements the dynamics of a simple damped pendulum (with mass, length, and damping set to 1, and gravity set to 10): $$\ddot\theta + \dot\theta + 10\sin\theta = u.$$  Write code into the following block that results in the pendulum_system variable being an *instance* of a Drake System that implements these dynamics using position and velocity as the state.

Hint: You can import `sin` from `numpy` or from `pydrake.math`. Depending on your version of `numpy`, it might print a warning, but this can be safely ignored.

In [ ]:
import numpy as np
from pydrake.all import LeafSystem, BasicVector, sin

class DampedPendulum(LeafSystem):
    def __init__(self):
        super().__init__()
        
        # Two state variables: θ (angle), ω (angular velocity)
        self.DeclareContinuousState(2)
        
        # One input: control torque u
        self.DeclareVectorInputPort("u", BasicVector(1))
        
        # One output: full state [θ, ω]
        self.DeclareVectorOutputPort("state", BasicVector(2), self.CopyStateOut)

    def CopyStateOut(self, context, output):
        output.SetFromVector(context.get_continuous_state_vector().CopyToVector())

    def DoCalcTimeDerivatives(self, context, derivatives):
        state = context.get_continuous_state_vector().CopyToVector()
        theta, omega = state  # θ and ω
        u = self.get_input_port(0).Eval(context)[0]  # Control input
        
        # Dynamics: θ'' + θ' + 10*sin(θ) = u
        theta_dot = omega
        omega_dot = -omega - 10 * sin(theta) + u
        
        derivatives.SetFromVector([theta_dot, omega_dot])

# Create an instance of the system
pendulum_system = DampedPendulum()



Drake [Systems](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_system.html) have methods that define the dynamics, like $$\dot{x} = f(t, x, u), \qquad y = g(t, x, u)$$ for the state dynamics, $f$, and system output function, $g$. Sometimes the equations depend on time, sometimes not. Sometimes they have extra parameters, sometimes not. Not all systems have inputs, and not all systems have states!

To make the input arguments to these methods more succinct, we collect $t, x, u$ and parameters into a single structure, which we call the [Context](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_context.html). Then all of the methods for the dynamical system can be called using that context, e.g.: $$\dot{x} = f(\text{context}), \qquad y = g(\text{context}).$$ We'll use it often throughout the course.

In [2]:
if pendulum_system is not None:
    context = pendulum_system.CreateDefaultContext()
    print(context)

    # Set the time and state
    context.SetTime(4.67)
    context.SetContinuousState([2.1, 3.4])
    print(context)

    # Set an input port to have a fixed value
    # (alternatively, we could connect it to the output port of another system).
    pendulum_system.get_input_port(0).FixValue(context, [7.89])
    print(f"Input port 0 value: {pendulum_system.get_input_port(0).Eval(context)}")

::_ Context
------------
Time: 0
States:
  2 continuous states
    0 0


::_ Context
------------
Time: 4.67
States:
  2 continuous states
    2.1 3.4


Input port 0 value: [7.89]


## Autograding
You can check your work by running the following cell:

In [3]:
from underactuated.exercises.grader import Grader
from underactuated.exercises.intro.test_drake_systems import TestDrakeSystems

Grader.grade_output([TestDrakeSystems], [locals()], "results.json")
Grader.print_test_results("results.json")

Total score is 2/2.

Score for test_dynamics (underactuated.exercises.intro.test_drake_systems.TestDrakeSystems.test_dynamics) is 1/1.

Score for test_input_and_state (underactuated.exercises.intro.test_drake_systems.TestDrakeSystems.test_input_and_state) is 1/1.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=52e7e101-429f-4aef-a373-e4cca7980cfe' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>